In [1]:
import mitiq

mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [2]:
frontend = "qiskit"

In [3]:
from mitiq import benchmarks

circuit = benchmarks.generate_rb_circuits(
  n_qubits=1, num_cliffords=2, return_type=frontend,
)[0]

print(circuit)

   ┌──────────┐┌────┐┌───┐┌──────┐
q: ┤ Ry(-π/2) ├┤ √X ├┤ S ├┤ √Xdg ├
   └──────────┘└────┘└───┘└──────┘


In [4]:
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    # We add a simple noise model to simulate a noisy backend.
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [5]:
# Compute the expectation value of the |0⟩⟨0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}")

Error without mitigation: 0.02614


In [6]:
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

4 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.


In [7]:
print(reps[0])

q_0: ───X^-0.5─── = 1.010*(q_0: ───X^-0.5───)-0.003*(q_0: ───X^-0.5───X───)-0.003*(q_0: ───X^-0.5───Y───)-0.003*(q_0: ───X^-0.5───Z───)


In [8]:
from mitiq import pec

sampled_circuits = pec.generate_sampled_circuits(circuit, representations=reps)

print(sampled_circuits[0])

   ┌──────────┐┌────┐┌───┐┌──────┐
q: ┤ Ry(-π/2) ├┤ √X ├┤ S ├┤ √Xdg ├
   └──────────┘└────┘└───┘└──────┘


In [9]:
print(sampled_circuits[1])

   ┌──────────┐┌────┐┌───┐┌──────┐
q: ┤ Ry(-π/2) ├┤ √X ├┤ S ├┤ √Xdg ├
   └──────────┘└────┘└───┘└──────┘


In [10]:
print(sampled_circuits[2])

   ┌──────────┐┌────┐┌───┐┌──────┐
q: ┤ Ry(-π/2) ├┤ √X ├┤ S ├┤ √Xdg ├
   └──────────┘└────┘└───┘└──────┘


In [11]:
pec_value = pec.execute_with_pec(circuit, execute, representations=reps)

print(f"Error without PEC: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC:    {abs(ideal_value - pec_value) :.5f}")

Error without PEC: 0.02614
Error with PEC:    0.02037


In [12]:
results = [execute(circuit) for circuit in sampled_circuits]

In [17]:
pec_value_combined = pec.combine_results(results, 1 , [1,1,1])

In [18]:
pec_value_combined

0.9738618731498718

In [15]:
print(f"Error without PEC: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC:    {abs(ideal_value - pec_value_combined) :.5f}")

Error without PEC: 0.02614
Error with PEC:    0.04931


In [16]:
pec_value

0.9796274065969119